In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 14:35:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
green_df = spark.read.parquet('../data/taxi_tripdata/green_tripdata_2022-*.parquet')
yellow_df = spark.read.parquet('../data/taxi_tripdata/yellow_tripdata_2022-*.parquet')

In [5]:
green_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
# rename columns
green_df = green_df.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
                   .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')
yellow_df = yellow_df.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
                     .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [7]:
yellow_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [10]:
# now we can see that the arrangment of columns in both dataframes are different.
# we need to arrange them.....
yellow_cols = set(yellow_df.columns)
common_cols = [col for col in green_df.columns if col in yellow_cols]
yellow_df = yellow_df.select(common_cols).withColumn('service_type', F.lit('yellow'))
green_df = green_df.select(common_cols).withColumn('service_type', F.lit('green'))

In [13]:
taxi_tripdata_df = green_df.unionAll(yellow_df)
taxi_tripdata_df.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green|  567297|
|      yellow|26144654|
+------------+--------+



In [14]:
taxi_tripdata_df.repartition(4).write.parquet('../data/taxi_tripdata/all_data_part.parquet')